## Notebook 2 - Combine Data and Add Columns
The purpose of this notebook is threefold:  
1. Convert pandas timestamp to a datetime aware object adjusted for time zone.
1. Add some features to the data - country and individual date components: year, month, hour (organization's local time) and day of week (where 0 = Monday, 1 = Tuesday, etc.). 
1. Join all the dataframes into one.

In [ ]:
import pandas as pd

### Write code to adjust dates and times according to DST.

In [ ]:
from datetime import datetime
import pytz
# function that replaces pandas timestamp with timezone adjusted datetime aware object
def convert_to_datetime_aware(date, tz=None):
    '''
    Parameters: 
        date - pandas UTC timestamp , 
        tz - a pytz time zone to convert to (default is None)
    
    Returns:
        datetime aware object in specified time zone (UTC if None)
    
    '''
    utc_zone = pytz.timezone('UTC')
    date = date.to_pydatetime().replace(tzinfo=utc_zone)
    
    if tz:
        return date.astimezone(tz)
    
    return date

In [ ]:
def add_date_features(df):
    df['year'] = df.date.apply(lambda x: x.year)
    df['month'] = df.date.apply(lambda x: x.month)
    df['weekday'] = df.date.apply(lambda x: x.weekday())
    df['hour'] = df.date.apply(lambda x: x.hour)
    return df

In [ ]:
# get the time zones
et_zone = pytz.timezone('US/Eastern')
ct_zone = pytz.timezone('US/Central')
mt_zone = pytz.timezone('US/Mountain')
pt_zone = pytz.timezone('US/Pacific')

In [ ]:
# import individual organization's tweets from the pickle files
df_sfspca = pd.read_pickle('../data/sfspca_tweets.p')
df_pspca = pd.read_pickle('../data/pspca_tweets.p')
df_houston = pd.read_pickle('../data/houston_tweets.p')
df_texas = pd.read_pickle('../data/texas_tweets.p')
df_tulsa = pd.read_pickle('../data/tulsa_tweets.p')
df_richmond = pd.read_pickle('../data/richmond_tweets.p')
df_ontario = pd.read_pickle('../data/ontario_tweets.p')
df_alberta = pd.read_pickle('../data/alberta_tweets.p')
df_bc = pd.read_pickle('../data/bc_tweets.p')

In [ ]:
df_sfspca.head(50)

In [ ]:
# convert date column of each dataframe to local time zone datetime aware object
df_sfspca['date'] = df_sfspca.date.apply(convert_to_datetime_aware, args=(pt_zone,))
df_pspca['date'] = df_pspca.date.apply(convert_to_datetime_aware, args=(et_zone,))
df_houston['date'] = df_houston.date.apply(convert_to_datetime_aware, args=(ct_zone,))
df_texas['date'] = df_texas.date.apply(convert_to_datetime_aware, args=(ct_zone,))
df_tulsa['date'] = df_tulsa.date.apply(convert_to_datetime_aware, args=(ct_zone,))
df_richmond['date'] = df_richmond.date.apply(convert_to_datetime_aware, args=(et_zone,))
df_ontario['date'] = df_ontario.date.apply(convert_to_datetime_aware, args=(et_zone,))
df_alberta['date'] = df_alberta.date.apply(convert_to_datetime_aware, args=(mt_zone,))
df_bc['date'] = df_bc.date.apply(convert_to_datetime_aware, args=(pt_zone,))

In [ ]:
# join dataframes based on country to add the country column
df_usa = pd.concat([df_sfspca, df_pspca, df_houston, df_texas, df_tulsa, df_richmond], ignore_index=True)
df_canada = pd.concat([df_ontario, df_alberta, df_bc], ignore_index=True)

In [ ]:
# add country to each dataframe
df_usa['country'] = 'usa'
df_canada['country'] = 'canada'
df_usa.shape, df_canada.shape

In [ ]:
# combine into one dataframe
df = pd.concat([df_usa, df_canada], ignore_index=True)
df.shape

In [ ]:
# add columns for year, month, weekday, and hour
df = add_date_features(df)
df.shape

In [ ]:
# change name of date column to local_datetime
df.rename(columns={'date':'local_datetime'}, inplace=True)

In [ ]:
df.to_pickle('../data/all_tweets.p')